In [426]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image
import numpy as np
import json
from skimage.draw import line

In [427]:
class Weather():
    def __init__(self, weather_img = "weather.png" , weather_json = "weather.json"):
        self.weather_matrix = self.to_matrix(weather_img).astype(np.int8)
        
        self.bottom_right, self.top_left = self.coordinates(weather_json)
        
    def coordinates(self, weather_json):
        weather_coordinates = json.load(open('weather.json'))
        
        return [weather_coordinates["bottom_right"]["latitude"], weather_coordinates["bottom_right"]["longitude"]] \
                 , [ weather_coordinates["top_left"]["latitude"], weather_coordinates["top_left"]["longitude"]]
    def br(self):
        return self.bottom_right
    def tl(self):
        return self.top_left
    
    def to_matrix(self, weather_img):
        
        weather_matrix = np.array(matplotlib.image.imread(weather_img)[: ,: , 1])
        un = np.unique(weather_matrix[0], axis = 0)
        
        def map_green(x):
            return (np.isclose(x , un[1]))
        def map_yellow(x):
            return (np.isclose(x , un[2]))
        
        return weather_matrix*0 + map_green(weather_matrix)*1 + map_yellow(weather_matrix)*2
    
    def wmatrix(self):
        
        return self.weather_matrix
    
    def weight_between_points(self, p1, p2):
        
        assert (abs(self.bottom_right[0]) < abs(p1[0]) and abs(p1[0]) < abs(self.top_left[0])) and (abs(self.bottom_right[1]) < abs(p1[1]) and abs(p1[1]) < abs(self.top_left[1]))
        assert (abs(self.bottom_right[0]) < abs(p2[0]) and abs(p2[0]) < abs(self.top_left[0])) and (abs(self.bottom_right[1]) < abs(p2[1]) and abs(p2[1]) < abs(self.top_left[1]))
        
        p1_idx_x, p1_idx_y = self.coordinates_to_pixel_idx(p1)
        p2_idx_x, p2_idx_y = self.coordinates_to_pixel_idx(p2)

        weight_of_pixels_in_line = self.weather_matrix[line(p1_idx_x, p1_idx_y, p2_idx_x, p2_idx_y)]
                
        val_counts =  np.array(np.unique(weight_of_pixels_in_line, return_counts = True))
        
        weather = [0, 0, 0]
        for i in range(0, len(val_counts[0])):
            weather[val_counts[0][i]] = val_counts[1][i] 
        
        return weather
        
    def coordinates_to_pixel_idx(self, point):
        
        x_array = np.arange(self.top_left[0], self.bottom_right[0], (self.bottom_right[0]-self.top_left[0])/self.weather_matrix.shape[0])
        y_array = np.arange(self.top_left[1], self.bottom_right[1], (self.bottom_right[1]-self.top_left[1])/self.weather_matrix.shape[1])

        return  np.abs(x_array - point[0]).argmin(), np.abs(y_array - point[1]).argmin()

In [428]:
w = Weather()

In [436]:
#plt.figure(figsize = (20, 20))
#plt.imshow(w.wmatrix())

In [430]:
br = w.br()

In [431]:
tl = w.tl()

In [432]:
br

[21.94, -67.5]

In [433]:
tl

[55.78, -135.0]

In [434]:
w.weight_between_points([35.043794, -106.816311], [33.371758, -96.361231])

[140, 90, 49]